In [1]:
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import logging
import warnings
warnings.filterwarnings("ignore")

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Load datasets
data_path = "data"
file_names = {
    "bitcoin": "bitcoin.csv",
    "gold": "gold.csv",
    "google_trends": "google_trends.csv",
    "sp500": "sp500.csv",
    "treasury_3m": "treasury_3m.csv",
    "treasury_10y": "treasury_10y.csv",
    "copper": "copper.csv",
    "oil": "oil.csv",
    "unemployment": "unemployment.csv",
    "moex": "MOEX.csv",
    "sse": "SSE.csv",
    "stoxx": "STOXX_600.csv",
    "vix": "vix.csv",
    "spgsci": "spgsci.csv"
}

data = {}
for key, file in file_names.items():
    file_path = os.path.join(data_path, file)
    if os.path.exists(file_path):
        logging.info(f"Loading {file}")
        df = pd.read_csv(file_path)
        if "timestamp" not in df.columns:
            logging.warning(f"{file} skipped: no 'timestamp' column")
            continue
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df.replace({'.': np.nan}, inplace=True)
        for col in df.columns:
            if col != "timestamp":
                df[col] = pd.to_numeric(df[col], errors='coerce')
        data[key] = df
    else:
        logging.warning(f"{file} not found")

rename_map = {
    "bitcoin": {"Close": "bitcoin_close", "Open": "bitcoin_open", "High": "bitcoin_high", "Low": "bitcoin_low", "Volume": "bitcoin_volume"},
    "gold": {"Close": "gold_close", "Open": "gold_open", "High": "gold_high", "Low": "gold_low", "Volume": "gold_volume"},
    "oil": {"Close": "oil_close", "Open": "oil_open", "High": "oil_high", "Low": "oil_low", "Volume": "oil_volume"},
    "copper": {"price": "copper_price"},
    "google_trends": {"SPX": "google_spx", "ETF": "google_etf", "index fund": "google_index_fund", "sp500": "google_sp500"},
    "unemployment": {"Unemployment": "unemployment_rate"},
    "treasury_3m": {"Close": "treasury_3m"},
    "treasury_10y": {"Close": "treasury_10y"},
    "sp500": {"Close": "sp500_close"},
    "moex": {"Close": "moex_close"},
    "sse": {"Close": "sse_close"},
    "stoxx": {"Close": "stoxx_close"},
    "vix": {"Close": "vix_close"},
    "spgsci": {"Close": "spgsci_close"}
}

for key, renames in rename_map.items():
    if key in data:
        data[key] = data[key].rename(columns=renames)

logging.info("Merging all dataframes")
sp500 = data["sp500"]
all_data = sp500[["timestamp", "sp500_close"]]
for key, df in data.items():
    if key != "sp500":
        all_data = all_data.merge(df, on="timestamp", how="left")

all_data.sort_values("timestamp", inplace=True)
all_data.fillna(method="ffill", inplace=True)
all_data.dropna(inplace=True)

def add_lag_rolling_features(df, base_cols, lags=[1, 3, 7], windows=[3, 7]):
    for col in base_cols:
        for lag in lags:
            df[f"{col}_lag_{lag}"] = df[col].shift(lag)
        for window in windows:
            df[f"{col}_roll_mean_{window}"] = df[col].rolling(window).mean()
            df[f"{col}_roll_std_{window}"] = df[col].rolling(window).std()
    return df

key_features = ["sp500_close", "vix_close", "bitcoin_close", "oil_close"]
all_data = add_lag_rolling_features(all_data, key_features)
all_data.dropna(inplace=True)
all_data = all_data.loc[:, all_data.nunique() > 1]
logging.info("Final dataset shape with lags/rolls: %s", all_data.shape)

target = "sp500_close"
exclude_cols = ["timestamp", target]
exog_vars = [col for col in all_data.columns if col not in exclude_cols]


from statsmodels.tsa.stattools import adfuller

def adf_test(series, signif=0.05, name=''):
    result = adfuller(series.dropna(), autolag='AIC')
    p_value = result[1]
    print(f'ADF Test for {name}: p-value = {p_value}')
    return p_value < signif

def make_stationary(df, columns, signif=0.05):
    stationary_df = df.copy()
    for col in columns:
        if not adf_test(df[col], signif=signif, name=col):
            stationary_df[col] = df[col].diff().dropna()
            print(f'--> Differenced {col}')
        else:
            print(f'--> {col} is already stationary')
    return stationary_df

# Make target and exogenous variables stationary
all_data = make_stationary(all_data, [target] + exog_vars)
all_data.dropna(inplace=True)

results = []
start_date = pd.Timestamp("2023-01-01")
end_date = pd.Timestamp("2025-03-01")
current_date = start_date

while current_date <= end_date:
    logging.info(f"Backtest for month starting {current_date.strftime('%Y-%m-%d')}")
    next_month = current_date + pd.DateOffset(months=1)

    train = all_data[all_data["timestamp"] < current_date]
    test = all_data[(all_data["timestamp"] >= current_date) & (all_data["timestamp"] < next_month)]

    if len(train) < 100 or len(test) < 10:
        logging.warning("Skipping month due to insufficient data")
        current_date += pd.DateOffset(months=1)
        continue

    y_train = train[target]
    y_test = test[target]
    X_train = train[exog_vars]
    X_test = test[exog_vars]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    pca = PCA(n_components=0.95)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    best_aic = np.inf
    best_model = None
    best_order = None
    best_seasonal_order = None

    for p in [0, 1]:
        for q in [0, 1]:
            for P in [0, 1]:
                for Q in [0, 1]:
                    try:
                        model = SARIMAX(
                            y_train,
                            exog=X_train_pca,
                            order=(p, 1, q),
                            seasonal_order=(P, 0, Q, 12),
                            enforce_stationarity=False,
                            enforce_invertibility=False
                        )
                        model_fit = model.fit(disp=False)
                        if model_fit.aic < best_aic:
                            best_aic = model_fit.aic
                            best_model = model_fit
                            best_order = (p, 1, q)
                            best_seasonal_order = (P, 0, Q, 12)
                    except:
                        continue

    if best_model:
        forecast = best_model.forecast(steps=len(X_test_pca), exog=X_test_pca)
        r2 = r2_score(y_test, forecast)
        mae = mean_absolute_error(y_test, forecast)
        rmse = np.sqrt(mean_squared_error(y_test, forecast))

        results.append({
            "Backtest": len(results) + 1,
            "Train Size": len(train),
            "Test Start": current_date.strftime("%Y-%m-%d"),
            "R2": r2,
            "MAE": mae,
            "RMSE": rmse,
            "Order": best_order,
            "Seasonal Order": best_seasonal_order,
            "AIC": best_aic
        })

    current_date += pd.DateOffset(months=1)

results_df = pd.DataFrame(results)
os.makedirs("results", exist_ok=True)
results_df.to_csv("results/sarimax_backtest_lags_rolls.csv", index=False)
print(results_df)

2025-04-15 22:12:05,302 - INFO - Loading bitcoin.csv
2025-04-15 22:12:05,311 - INFO - Loading gold.csv
2025-04-15 22:12:05,314 - INFO - Loading google_trends.csv
2025-04-15 22:12:05,319 - INFO - Loading sp500.csv
2025-04-15 22:12:05,323 - INFO - Loading treasury_3m.csv
2025-04-15 22:12:05,327 - INFO - Loading treasury_10y.csv
2025-04-15 22:12:05,336 - INFO - Loading copper.csv
2025-04-15 22:12:05,342 - INFO - Loading oil.csv
2025-04-15 22:12:05,348 - INFO - Loading unemployment.csv
2025-04-15 22:12:05,355 - INFO - Loading MOEX.csv
2025-04-15 22:12:05,359 - INFO - Loading SSE.csv
2025-04-15 22:12:05,364 - INFO - Loading STOXX_600.csv
2025-04-15 22:12:05,370 - INFO - Loading vix.csv
2025-04-15 22:12:05,375 - INFO - Loading spgsci.csv
2025-04-15 22:12:05,381 - INFO - Merging all dataframes
2025-04-15 22:12:05,432 - INFO - Final dataset shape with lags/rolls: (794, 69)


ADF Test for sp500_close: p-value = 0.8926384143250452
--> Differenced sp500_close
ADF Test for bitcoin_open: p-value = 0.8601557186055624
--> Differenced bitcoin_open
ADF Test for bitcoin_high: p-value = 0.8520555320714517
--> Differenced bitcoin_high
ADF Test for bitcoin_low: p-value = 0.8631500122146292
--> Differenced bitcoin_low
ADF Test for bitcoin_close: p-value = 0.8702735690883003
--> Differenced bitcoin_close
ADF Test for bitcoin_volume: p-value = 0.12896449339619404
--> Differenced bitcoin_volume
ADF Test for gold_open: p-value = 0.16048295887033676
--> Differenced gold_open
ADF Test for gold_high: p-value = 0.10913514056787305
--> Differenced gold_high
ADF Test for gold_low: p-value = 0.17584704671779539
--> Differenced gold_low
ADF Test for gold_close: p-value = 0.1890621062818884
--> Differenced gold_close
ADF Test for gold_volume: p-value = 5.014545415299984e-07
--> gold_volume is already stationary
ADF Test for google_sp500: p-value = 0.15591318307506263
--> Differenced

2025-04-15 22:13:22,788 - INFO - Backtest for month starting 2023-01-01


ADF Test for oil_close_roll_std_7: p-value = 9.986726133888856e-08
--> oil_close_roll_std_7 is already stationary


2025-04-15 22:14:26,927 - INFO - Backtest for month starting 2023-02-01
2025-04-15 22:15:39,275 - INFO - Backtest for month starting 2023-03-01
2025-04-15 22:28:46,720 - INFO - Backtest for month starting 2023-04-01
2025-04-15 22:42:29,225 - INFO - Backtest for month starting 2023-05-01
2025-04-15 22:55:19,260 - INFO - Backtest for month starting 2023-06-01
2025-04-15 23:05:53,597 - INFO - Backtest for month starting 2023-07-01
2025-04-15 23:20:13,353 - INFO - Backtest for month starting 2023-08-01
2025-04-15 23:24:35,351 - INFO - Backtest for month starting 2023-09-01
2025-04-15 23:28:24,688 - INFO - Backtest for month starting 2023-10-01
2025-04-15 23:32:06,841 - INFO - Backtest for month starting 2023-11-01
2025-04-15 23:35:51,276 - INFO - Backtest for month starting 2023-12-01
2025-04-15 23:39:40,039 - INFO - Backtest for month starting 2024-01-01
2025-04-15 23:43:32,836 - INFO - Backtest for month starting 2024-02-01
2025-04-15 23:47:26,146 - INFO - Backtest for month starting 202

    Backtest  Train Size  Test Start        R2       MAE      RMSE      Order  \
0          1         242  2023-01-01  0.310115  2.696164  3.309849  (1, 1, 1)   
1          2         262  2023-02-01  0.419432  2.605865  2.976651  (1, 1, 1)   
2          3         281  2023-03-01  0.396395  2.731962  3.504855  (1, 1, 1)   
3          4         304  2023-04-01  0.322419  2.107529  2.484898  (1, 1, 1)   
4          5         323  2023-05-01  0.264513  2.456564  2.779244  (1, 1, 1)   
5          6         345  2023-06-01  0.274593  1.988084  2.457699  (1, 1, 1)   
6          7         366  2023-07-01 -0.004132  1.797990  2.203619  (1, 1, 1)   
7          8         386  2023-08-01  0.550689  1.792773  2.261382  (1, 1, 1)   
8          9         409  2023-09-01  0.355678  1.880415  2.475276  (1, 1, 1)   
9         10         429  2023-10-01  0.436617  2.113536  2.745268  (0, 1, 1)   
10        11         451  2023-11-01  0.237377  1.932172  2.602672  (0, 1, 1)   
11        12         472  20